<a href="https://colab.research.google.com/github/rtikyani/CS634_IBM_Fairness_Pipeline/blob/master/CS634_Project1_Team3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial summary of the pipeline in a markdown file:

# [**Summary.md**](https://github.com/rtikyani/CS634_IBM_Fairness_Pipeline/blob/master/README.md)



# Execute the credit decision pipeline that is detecting age bias and removing using the reweighting algorithm and explain the findings:

In [ ]:
#Part 2

#install all necessary libraries and modules
!pip install aif360
!pip install IPython
!pip install BlackBoxAuditing
!pip install wget


#Retrieve Necessary Files 
import os
os.chdir('/usr/local/lib/python3.6/dist-packages/aif360/data/raw/german')
!wget -q	https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
!wget -q	https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc


#import the Methods used from aif360
from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

#Loading dataset
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'] # ignore sex-related attributes
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

#Metric for original data set
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#Applying Reweighing Pre-Processing method to mitigate the bias

#Train with and transform the original training data
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

# Metric with the transformed training data

metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.169905


#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


# Findings:

[OUTPUT]: 

**Original training dataset**: Difference in mean outcomes between unprivileged and privileged groups = -0.169905 

**Transformed training dataset**: Difference in mean outcomes between unprivileged and privileged groups = 0.000000

**Explanation:**

Protected Attribute: Age
Attributes not accounted for: Sex & Personal Status


# **Using [Disparate Impact Remover](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.DisparateImpactRemover.html#aif360.algorithms.preprocessing.DisparateImpactRemover)** for detecting and removing bias.


In [2]:
#Part 3

#install all necessary libraries and modules
!pip install aif360
!pip install IPython
!pip install BlackBoxAuditing
!pip install wget


#Retrieve Necessary Files 
import os
os.chdir('/usr/local/lib/python3.6/dist-packages/aif360/data/raw/german')
!wget -q	https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
!wget -q	https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc


#import the Methods used from aif360
from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

from IPython.display import Markdown, display

#Loading dataset
dataset_orig = GermanDataset(
    protected_attribute_names=['age'],           # this dataset also contains protected
                                                 # attribute for "sex" which we do not
                                                 # consider in this evaluation
    privileged_classes=[lambda x: x >= 25],      # age >=25 is considered privileged
    features_to_drop=['personal_status', 'sex'] # ignore sex-related attributes
)

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

#Metric for original data set
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())


#Using Disparate Impact Remover PreProcessor Method  

#repair_level - Repair amount
#0.0 = no repair 
#1.0 is full repair.
DIR = DisparateImpactRemover(repair_level = 1.0)

#Run a repairer on the non-protected features and return the transformed dataset.
dataset_transf_DIR = DIR.fit_transform(dataset_orig_train)


dataset_transf_DIR_train, dataset_transf_DR_test, = dataset_transf_DIR.split([0.7], shuffle=True)

metric_DIR_train = BinaryLabelDatasetMetric(dataset_transf_DIR_train, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

display(Markdown("#### Transformed training dataset using Disparate Impact Remover PreProcessor Method"))

print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_DIR_train.mean_difference())


     |████████████████████████████████| 174kB 2.8MB/s 


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.095783


ModuleNotFoundError: ignored

# Findings using Disparate Impact Remover

The alternate method used was Disparate Impact Remover method. This is a different Pre-Processing technique which falls in the aif360 library. 

**Results:**

**Original training dataset:**
Difference in mean outcomes between unprivileged and privileged groups = -0.169905

**Transformed training dataset using Disparate Impact Remover PreProcessor Method:**
Difference in mean outcomes between unprivileged and privileged groups = -0.177184

Using the Disparate Impact Remover method, the difference in mean outcome between unprivileged and privileged groups was -0.177184 -- a **greater** difference than using the Reweighing method. 

This demonstrates that although multiple techniques can be utilized for PreProcessing data, not all will be optimal.The reason being the different methods have unique algorithmic approaches. Like the Reweighing method, the DIR method aims to increases group fairness but does this while prioritizing preserving rank-ordering within groups. On the other hand, the Reweighing technique calculates the weight of each combination of a group and label to ensure fairness. For this dataset, the Reweighing method proved to be optimal as it transformed the dataset and mitigated the fairness to reach a difference in mean outcome of 0.00. Since age 

